<a href="https://colab.research.google.com/github/JoanaEnes/Coursera_Capstone/blob/main/Segmenting_and_Clustering_Neighborhoods_in_Toronto_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis


import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests - this module helps us to download a web page
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

from bs4 import BeautifulSoup #import BeautifulSoup - this module helps in web scrapping.

print('Libraries imported.')

Libraries imported.


In [2]:
#get information from wikipedia

List_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikipedia_source = requests.get(List_url).text
soup = BeautifulSoup(wikipedia_source, 'xml')

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [4]:
# Combining the neighbourhoods with same Postalcode
df_clean = df.groupby(['PostalCode','Borough'], sort=False).agg(', '.join)
df_clean.reset_index(inplace=True)
df_clean

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
# get lat and long from PostalCodes

geolocation_df=pd.read_csv('https://cocl.us/Geospatial_data')
geolocation_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [6]:
# merge two dataframes
geolocation_df.rename(columns={'Postal Code':'PostalCode'},inplace=True)
geolocation_df.head()
geolocation_merged = pd.merge(geolocation_df, df_clean, on='PostalCode')
geolocation_merged.head()

,PostalCode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


In [39]:
#creating a df just with toronto
df_toronto = geolocation_merged[geolocation_merged['Borough'].str.contains('Toronto',regex=False)]
# print number of rows of dataframe
df_toronto.shape[0]


39

In [8]:
# getting Toronto postal code
address = 'Toronto , ON'
geolocator = Nominatim(user_agent="Toronto")
location = geolocator.geocode(address)
latitude_toronto = location.latitude
longitude_toronto = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [9]:
#creating and plotting toronto map

toronto_map = folium.Map(location=[43.6534817,-79.3839347],zoom_start=12)

for lat,lng,borough,neighbourhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(toronto_map)
toronto_map

In [11]:
#df_clustering = df_toronto['PostalCode'], df_toronto['Borough'], df_toronto['Neighborhood']

df_clustering = df_toronto.drop(['PostalCode', 'Borough','Neighborhood'], 1)

# run k-means clustering

kmeans = KMeans(n_clusters = 5,random_state=0).fit(df_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
df_clustering.insert(0, 'Cluster Labels', kmeans.labels_)

df_clustering

,Cluster Labels,Latitude,Longitude
37,4,43.676357,-79.293031
40,4,43.685347,-79.338106
41,4,43.679557,-79.352188
42,4,43.668999,-79.315572
43,4,43.659526,-79.340923
44,2,43.728020,-79.388790
45,2,43.712751,-79.390197
46,2,43.715383,-79.405678
47,2,43.704324,-79.388790
48,2,43.689574,-79.383160


In [13]:
df_data = pd.merge(df_clustering, df_toronto, on=['Latitude','Longitude'] )
df_data

,Cluster Labels,Latitude,Longitude,PostalCode,Borough,Neighborhood
0,4,43.676357,-79.293031,M4E,East Toronto,The Beaches
1,4,43.685347,-79.338106,M4J,East YorkEast Toronto,The Danforth East
2,4,43.679557,-79.352188,M4K,East Toronto,"The Danforth West, Riverdale"
3,4,43.668999,-79.315572,M4L,East Toronto,"India Bazaar, The Beaches West"
4,4,43.659526,-79.340923,M4M,East Toronto,Studio District
5,2,43.728020,-79.388790,M4N,Central Toronto,Lawrence Park
6,2,43.712751,-79.390197,M4P,Central Toronto,Davisville North
7,2,43.715383,-79.405678,M4R,Central Toronto,North Toronto West
8,2,43.704324,-79.388790,M4S,Central Toronto,Davisville
9,2,43.689574,-79.383160,M4T,Central Toronto,"Moore Park, Summerhill East"


In [ ]:
toronto_clusters = folium.Map(location=[43.6534817,-79.3839347],zoom_start=12)

for lat,lng,borough,neighbourhood in zip(df_toronto['Latitude'],df_toronto['Longitude'],df_toronto['Borough'],df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(toronto_map)
toronto_map

In [20]:
# create map
map_clusters = folium.Map(location=[43.6534817,-79.3839347], zoom_start=11)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_data['Latitude'], df_data['Longitude'], df_data['Neighborhood'], df_data['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters